In [ ]:
%%capture
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from __future__ import print_function, division
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import albumentations

In [ ]:
class CustomDatset:
    def __init__(self,image_path,targets,resize,augmentations=None):
        self.image_path = image_path
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
    def __len__(self):
      return len(self.targets)

    def __getitem__(self,item):
      image = io.imread(self.image_path[item])  
      targets = self.targets[item]
      image = transform.resize(image,self.resize)
      image = np.array(image)

      if self.augmentations is not None:
        augmented = self.augmentations(image=image)
        image=augmented['image']
     ## C x H x W instead of H x w x C   
      image = np.transpose(image,(2,0,1)).astype(np.float32)

      return{
          'image':torch.tensor(image,dtype=torch.float),
          'target':torch.tensor(targets,dtype=torch.long)
      }

In [ ]:
def train(data_loader,model,optimizer,device):

  model.train()

  for data in data_loader:
    inputs = data['image']
    targets = data['target']

    inputs = inputs.to(device, dtype=torch.float)
    targets = targets.to(device, dtype=torch.float)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = torch.sqrt(nn.MSELoss()(outputs, targets.view(-1,1)))
    loss.backward()
    optimizer.step()

def evaluate(data_loader,model,device):

  model.eval()
  
  final_targets = []
  final_outputs = []
  
  with torch.no_grad():

    for data in data_loader:
      inputs = data['image']
      targets = data['target']
      inputs = inputs.to(device, dtype=torch.float)
      targets = targets.to(device, dtype=torch.float)

      output = model(inputs)

      targets = targets.detach().cpu().numpy().tolist()
      output = output.detach().cpu().numpy().tolist()
      final_outputs.extend(output)
      final_targets.extend(targets)  
  return final_outputs,final_targets    

In [ ]:
def predict(data_loader,model,device):

  
  final_outputs = []
  
  with torch.no_grad():

    for data in data_loader:
      inputs = data['image']
      
      inputs = inputs.to(device, dtype=torch.float)
      

      output = model(inputs)

      
      output = output.detach().cpu().numpy().tolist()
      final_outputs.extend(output)
       
  return final_outputs   

In [ ]:
! pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
def get_model():
  model = EfficientNet.from_pretrained('efficientnet-b0')
  model._fc = nn.Linear(in_features=1280, out_features=1, bias=True)

  return model

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

data_path = '../input/petfinder-pawpularity-score'
device = 'cuda'
epochs = 7
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

images = df.Id.values.tolist()
images = [os.path.join(data_path,'train',i + '.jpg') for i in images] 

targets = df.Pawpularity.values

model = get_model()
model.to(device)
mean = (0.5150, 0.4802, 0.4424) 
std =  (0.2326, 0.2310, 0.2314)

aug = albumentations.Compose(
    [
     albumentations.Normalize(mean,std,max_pixel_value=255.0,always_apply=True)
    ]
)

train_images, valid_images,train_targets, valid_targets = train_test_split(images,targets,test_size = 0.8,random_state = 42)

train_dataset = CustomDatset(image_path = train_images,targets = train_targets,resize = (224,224),augmentations=aug)
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers= 4,pin_memory=True) 
valid_dataset = CustomDatset(image_path = valid_images,targets =valid_targets,resize = (224,224),augmentations=aug)
valid_loader = DataLoader(valid_dataset,batch_size=16,shuffle=False,num_workers= 4,pin_memory=True) 

optimizer = torch.optim.Adam(model.parameters(),lr=5e-4)

for epoch in range(epochs):
  train(train_loader,model,optimizer,device=device)
  prediction,valid_targets = evaluate(valid_loader,model,device=device)
  RMSE = np.sqrt(metrics.mean_squared_error(valid_targets,prediction))

  print(f'Epoch = {epoch}   valid RMSE={RMSE}')


In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

data_path = '../input/petfinder-pawpularity-score'
device = 'cuda'
df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

test_images = df.Id.values.tolist()
test_images = [os.path.join(data_path,'test',i + '.jpg') for i in test_images] 

mean = (0.5150, 0.4802, 0.4424) 
std =  (0.2326, 0.2310, 0.2314)

aug = albumentations.Compose(
    [
     albumentations.Normalize(mean,std,max_pixel_value=255.0,always_apply=True)
    ]
)

test_dataset =  CustomDatset(image_path = test_images,targets = np.ones(len(test_images)),resize = (224,224),augmentations=aug)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False,num_workers=4) 


test_predictions = predict(test_loader,model,device)

In [ ]:
test_predictions

In [ ]:
from itertools import chain
import math
test_predictions = list(chain.from_iterable(test_predictions))

In [ ]:
submission1 = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
submission1['Pawpularity'] = test_predictions

In [ ]:
submission1.to_csv('submission.csv')